In [20]:
import tensorflow as tf
from tensorflow.keras.models import load_model
# load model
model = load_model('diabetesHRV.h5')

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                200       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 21        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [22]:
import pandas as pd
test_dataset = pd.read_csv('swell_testing_data.csv') #load test data
train_stats = pd.read_csv('model_stats.csv') # load train stats to normalize data
def labelDataset(row):
#     label the dataset 1- diabetic , 0- non-diabetic
    if row['label'] =="healthy":
        return  1
    else:
        return 0
    
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

# transform data
test_dataset['label'] = test_dataset.apply (lambda row: labelDataset(row), axis=1)
test_labels = test_dataset['label']
test_data = test_dataset.drop(['label'],axis =1)
normed_test_data = norm(test_data)

In [23]:
# evaluate model
score = model.evaluate(normed_test_data, test_labels, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

accuracy: 87.69%


In [24]:
import numpy as np
predictions = model.predict(normed_test_data)
result = np.zeros(len(predictions))
for p in range(len(predictions)):
    if predictions[p] > 0.5:
        result[p]= 1
    else:
        result[p]=0
result = result.transpose()
# confusion = tf.math.confusion_matrix(labels=test_labels, predictions=test_labels, num_classes=2)

In [25]:
confusion = tf.math.confusion_matrix(labels=test_labels, predictions=result, num_classes=2)

In [26]:
confusion

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[17204,  1671],
       [ 3382, 18776]])>